In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plotly.graph_objects as go
import h5py
import time

In [3]:
#file = h5py.File('/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/module-3/tpc_12-packet-2023_02_22_12_45_CET_module-3_events.h5', 'r')
file = h5py.File('/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/module-0/datalog_2021_04_04_00_41_40_CEST_module-0_events_60s_to_360s.h5', 'r')
#file = h5py.File('/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/module-0/datalog_2021_04_04_00_41_40_CEST_module-0_events_60s_to_150s_cosmics_veto.h5', 'r')

In [47]:
large_clusters = file['large_clusters']
small_clusters = file['small_clusters']

t_min = large_clusters['t_min']
mask = np.array([100,20,-30,-2,1,2,5,6,1000])[:, np.newaxis] > np.array([100,200,300,400,500,600,700,800,900])
mask

array([[False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [50]:
(np.sum(mask, axis=0)).astype('bool')

array([False, False, False, False, False, False, False, False, False])

In [51]:
np.array([0,2,1,3,0,55,4,0]).astype(bool)

array([False,  True,  True,  True, False,  True,  True, False])

In [32]:
true_indices = np.where(mask, np.arange(mask.shape[1]), np.nan)
true_indices

array([[nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [nan, nan,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [nan, nan, nan,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [nan, nan,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [nan, nan, nan, nan, nan,  5.,  6.,  7.,  8.],
       [nan, nan, nan, nan, nan, nan,  6.,  7.,  8.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.]])

In [14]:
np.where(np.array([1,2,3,0,1,2,5,6,0])[:, np.newaxis] < np.array([1,2,3,4,5,6,7,8,9]), axis=1)

TypeError: where() got an unexpected keyword argument 'axis'

In [3]:
small_clusters_near_cosmic = file['small_clusters_near_cosmic']
small_clusters_not_near_cosmic = file['small_clusters_not_near_cosmic']
large_clusters = file['large_clusters']

In [4]:
print('Number of small clusters near large clusters = ', len(small_clusters_near_cosmic))
print('Number of small clusters not near large clusters = ', len(small_clusters_not_near_cosmic))

Number of small clusters near large clusters =  303
Number of small clusters not near large clusters =  344893


In [6]:
len(small_clusters_near_cosmic)/(len(small_clusters_near_cosmic)+len(small_clusters_not_near_cosmic)) * 100

0.08777621988667308